<a href="https://colab.research.google.com/github/peisuke/ml-works/blob/main/4/4_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 4.3 バリデーションを用いたパラメータのチューニング
- 目的
    - 機械学習モデルをチューニングする際のプロセスについて学ぶ
    - テストデータを見ずに検証データを用いてチューニングする流れを確認する
- 内容
    - これまでと同様の学習
    - テストデータに対する誤ったチューニング
    - Holdoutを用いたチューニング
    - Cross validationを用いたチューニング
- 参考
    - 本コードは以下のサイトを参考にしました
    - https://qiita.com/tomov3/items/039d4271ed30490edf7b

In [1]:
# 必要なライブラリの import
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import cross_val_score 
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC

In [2]:
# データのロード
iris = load_iris()

# データの分割
X_train, X_test, y_train, y_test = train_test_split(iris.data, iris.target, test_size=0.4, random_state=0)

### 初期実験
- まずは、前の実験と同じように学習データとテストデータに分離、
- 学習データを使い学習し、テストデータで精度を検証

In [3]:
# これまで通りにtraining set を用いて学習し、test setで評価
svm = SVC()
svm.fit(X_train, y_train)
score = svm.score(X_test, y_test)
print('Test set score: {}'.format(score))

Test set score: 0.9333333333333333


### よくある間違い
- テストデータの精度を上げる際に、パラメータを設定→テスト精度をチェックをしてしまうこと
- 見かけ上の精度は良くなるが、テストデータに特化したパラメータが得られてしまう

In [4]:
# 以下の数値をSVCのパラメータ（gamma, C）に入れて実験
param_list = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1, 1, 5, 10, 20, 50, 100, 500, 1000, 5000]

best_score = 0
best_parameters = {}

for gamma in param_list:
    for C in param_list:
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)

        # テストデータのスコアを計算
        score = svm.score(X_test, y_test)

        # 最も良いスコアのパラメータとスコアを更新
        if score > best_score:
            best_score = score
            best_parameters = {'gamma' : gamma, 'C' : C}

# 最もテストデータに対する結果の良かったパラメータを表示
print('Best score: {}'.format(best_score))
print('Best parameters: {}'.format(best_parameters))

Best score: 0.9666666666666667
Best parameters: {'gamma': 0.0001, 'C': 5000}


### Holdoutを利用した検証
- 学習データを更に、訓練用・検証用の２つに分ける
- 片方で学習をし、もう片方で精度をチェック
- こうすることでテストデータを見ずに良いパラメータを探索できる
- ただし、検証データ量が少ない場合、誤ったパラメータが選択されてしまう場合がある

In [5]:
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, random_state=0)
X_trainval = np.concatenate((X_train, X_val))
y_trainval = np.concatenate((y_train, y_val))

print('Size of trainings set: {}, validation set: {}, test set: {}'.format(X_train.shape, X_val.shape, X_test.shape))

Size of trainings set: (67, 4), validation set: (23, 4), test set: (60, 4)


In [6]:
param_list = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1, 1, 5, 10, 20, 50, 100, 500, 1000, 5000]

best_score = 0
best_parameters = {}

for gamma in param_list:
    for C in param_list:
        svm = SVC(gamma=gamma, C=C)
        svm.fit(X_train, y_train)

        # 検証データを用いてscore を計算する
        score = svm.score(X_val, y_val)
        if score > best_score:
            best_score = score
            best_parameters = {'gamma' : gamma, 'C' : C}

# 検証データに対して最も性能の良かったパラメータを用いて、全学習データを用いて再学習
svm = SVC(**best_parameters)
svm.fit(X_trainval, y_trainval)

# テストデータによる評価は，ここで初めて行われる
test_score = svm.score(X_test, y_test)

print('Best score on validation set: {}'.format(best_score))
print('Best parameters: {}'.format(best_parameters))
print('Test set score with best parameters: {}'.format(test_score))

Best score on validation set: 1.0
Best parameters: {'gamma': 0.0001, 'C': 500}
Test set score with best parameters: 0.9333333333333333


### Cross validationを利用した検証
- Holdoutは検証データ量が少ない場合に誤ったパラメータが選択されてしまう場合があった
- Cross Validationを用いて、データをK個（以下では5個）に分割して、それぞれに対して精度評価
- 精度の平均が最も良いパラメータを採用
- 時間は掛かるものの、最も良い検証が可能

In [7]:
param_list = [0.0001, 0.0005, 0.001, 0.005, 0.01, 0.1, 1, 5, 10, 20, 50, 100, 500, 1000, 5000]

best_score = 0
best_parameters  = {}

for gamma in param_list:
    for C in param_list:
        svm = SVC(gamma=gamma, C=C)

        # cross_val_score() を用いてクロスバリデーション
        scores = cross_val_score(svm, X_trainval, y_trainval, cv=5)

        # K個の評価値の平均を用いる
        score = np.mean(scores)
        if score > best_score:
            best_score = score
            best_parameters = {'gamma' : gamma, 'C' : C}

# 検証データに対して最も性能の良かったパラメータを用いて、全学習データを用いて再学習
svm = SVC(**best_parameters)
svm.fit(X_trainval, y_trainval)

# テストデータによる評価は，ここで初めて行われる
test_score = svm.score(X_test, y_test)

print('Best score on validation set: {}'.format(best_score))
print('Best parameters: {}'.format(best_parameters))
print('Test set score with best parameters: {}'.format(test_score))

Best score on validation set: 0.9888888888888889
Best parameters: {'gamma': 0.0001, 'C': 5000}
Test set score with best parameters: 0.9666666666666667
